In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from math import pi

# ---------- Utility functions ----------
def sample_lorentzian(gamma, size):
    return gamma * np.random.standard_cauchy(size=size)

def heun_step_thetas(thetas, omegas, Ks, dt):
    Z = np.mean(np.exp(1j * thetas))
    R = np.abs(Z)
    phi = np.angle(Z)
    theta_dot = omegas + Ks * R * np.sin(phi - thetas)
    thetas_pred = thetas + dt * theta_dot
    Z_pred = np.mean(np.exp(1j * thetas_pred))
    R_pred = np.abs(Z_pred)
    phi_pred = np.angle(Z_pred)
    theta_dot_pred = omegas + Ks * R_pred * np.sin(phi_pred - thetas_pred)
    thetas_new = thetas + dt * 0.5 * (theta_dot + theta_dot_pred)
    return thetas_new

def simulate_microscopic_init(N, K1, K2, p, gamma, seed=None):
    if seed is not None:
        np.random.seed(seed)
    types = np.random.rand(N) < p
    Ks = np.where(types, K2, K1)
    omegas = sample_lorentzian(gamma, N)
    thetas = 2 * pi * np.random.rand(N)
    z1_0 = np.mean(np.exp(1j * thetas[Ks == K1])) if np.any(Ks == K1) else 0.0
    z2_0 = np.mean(np.exp(1j * thetas[Ks == K2])) if np.any(Ks == K2) else 0.0
    z1_2_0= np.mean(np.exp(1j * 2 * thetas[Ks == K1])) if np.any(Ks == K1) else 0.0
    z2_2_0= np.mean(np.exp(1j * 2 * thetas[Ks == K1])) if np.any(Ks == K1) else 0.0
    z1_3_0= np.mean(np.exp(1j * 3 * thetas[Ks == K1])) if np.any(Ks == K1) else 0.0
    z2_3_0= np.mean(np.exp(1j * 3 * thetas[Ks == K1])) if np.any(Ks == K1) else 0.0
    kappa1_init = z1_2_0 - z1_0**2
    kappa2_init = z2_2_0 - z2_0**2
    varkappa1_init = (z1_3_0 - 3*z1_2_0*z1_0 + 2*z1_0**3)/2
    varkappa2_init = (z2_3_0 - 3*z2_2_0*z2_0 + 2*z2_0**3)/2
    return thetas, omegas, Ks, z1_0, z2_0, kappa1_init, kappa2_init, varkappa1_init, varkappa2_init

def chemical_reaction_tau_leaping(types, Ks, K1, K2, r1, r2, dt):
    """
    使用tau-leaping算法处理化学反应
    types: 当前类型数组 (True=正/K2, False=负/K1)
    Ks: 耦合强度数组
    r1, r2: 反应速率
    dt: 时间步长
    返回: 更新后的types和Ks
    """
    N = len(types)
    n_positive = np.sum(types)
    n_negative = N - n_positive
    
    # 计算反应事件数（泊松分布）
    n_pos_to_neg = np.random.poisson(r1 * n_positive * dt)
    n_neg_to_pos = np.random.poisson(r2 * n_negative * dt)
    
    # 限制不超过可用个体数
    n_pos_to_neg = min(n_pos_to_neg, n_positive)
    n_neg_to_pos = min(n_neg_to_pos, n_negative)
    
    # 执行转换
    if n_pos_to_neg > 0:
        pos_indices = np.where(types)[0]
        convert_indices = np.random.choice(pos_indices, n_pos_to_neg, replace=False)
        types[convert_indices] = False
        Ks[convert_indices] = K1  # 更新耦合强度
    
    if n_neg_to_pos > 0:
        neg_indices = np.where(~types)[0]
        convert_indices = np.random.choice(neg_indices, n_neg_to_pos, replace=False)
        types[convert_indices] = True
        Ks[convert_indices] = K2  # 更新耦合强度
    
    return types, Ks

def simulate_microscopic(thetas, omegas, Ks, K1, K2, p0, r1, r2, dt, t_max):
    times = np.arange(0, t_max, dt)
    N = len(thetas)
    R_ts = np.zeros_like(times)
    R1_ts = np.zeros_like(times)
    R2_ts = np.zeros_like(times)
    p_ts = np.zeros_like(times)  # 记录p随时间变化
    Z_ts = np.zeros(len(times), dtype=complex)
    
    # 初始化类型数组
    types = (Ks == K2)
    p_ts[0] = np.mean(types)
    
    for i, t in enumerate(times):
        if i > 0:  # 跳过初始时间步
            # 化学反应
            types, Ks = chemical_reaction_tau_leaping(types, Ks, K1, K2, r1, r2, dt)
            p_ts[i] = np.mean(types)
        
        mask1 = (Ks == K1)
        mask2 = (Ks == K2)
        
        Z = np.mean(np.exp(1j * thetas))
        Z1 = np.mean(np.exp(1j * thetas[mask1])) if np.any(mask1) else 0
        Z2 = np.mean(np.exp(1j * thetas[mask2])) if np.any(mask2) else 0
        
        R_ts[i] = np.abs(Z)
        R1_ts[i] = np.abs(Z1)
        R2_ts[i] = np.abs(Z2)
        Z_ts[i] = Z
        
        # Kuramoto动力学
        thetas = heun_step_thetas(thetas, omegas, Ks, dt)
    
    return times, R_ts, R1_ts, R2_ts, Z_ts, p_ts

# ---------- Reduced Eq.(11) with time-dependent p(t) ----------


# ---------- 模拟函数 ----------
def run_simulation():
    """
    同时运行微观与还原模拟。
    还原模型部分现在返回 z1, z2, kappa1, kappa2, varkappa1, varkappa2。
    """
    # --- initialize 微观模拟 ---
    thetas, omegas, Ks, z1_0, z2_0, kappa1_init, kappa2_init, varkappa1_init, varkappa2_init = simulate_microscopic_init(
        N, K1, K2, p0, gamma, seed
    )
    # --- simulate 微观 ---
    times_m, Rm, R1m, R2m, Zm, p_micro = simulate_microscopic(
        thetas, omegas, Ks, K1, K2, p0, r1, r2, dt, t_max
    )
    # --- 返回所有模拟结果 ---
    return {
        # 微观模拟
        'times_m': times_m,
        'Rm': Rm,
        'R1m': R1m,
        'R2m': R2m,
        'Zm': Zm,
        'p_micro': p_micro,
        # 参数信息
        'params': {
            'N': N,
            'K1': K1,
            'K2': K2,
            'p0': p0,
            'gamma': gamma,
            'r1': r1,
            'r2': r2,
            'p_eq': p_eq,
        },
    }


N = 51200
K1, K2 = -5, 0.5
gamma = 0.05
p0 = 0.95  # 初始p值
p_s = 0.95 # 稳态p值
r1 = 0.03
r2 = (p_s * r1) / (1 - p_s)
dt = 0.02
t_max = 200.0
seed = 12345

results = run_simulation()